# Load data

In [1]:
import os
os.environ["NUMBA_ENABLE_CUDASIM"] = "1"
os.environ["NUMBA_CUDA_DEBUGINFO"] = "1"

In [2]:
import pubchempy as pcp
import requests
import json
import numpy as np
import pandas as pd
pd.set_option('display.max_rows',200,'display.max_columns',50)
import csv
import time
import pickle
from tqdm import tqdm
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import QED
from rdkit.Chem.Descriptors import MolWt
from rdkit.Chem import RDConfig
import os
import sys
sys.path.append(os.path.join(RDConfig.RDContribDir,'SA_Score'))
from sascorer import calculateScore
import pubchempy as pcp
from sklearn.model_selection import train_test_split
import numba as nb
from scipy import sparse

In [ ]:
root = "/home2/glee/Drug_Discovery_Research"
data_path = os.path.join(root, "data")
data = pd.read_csv(os.path.join(data_path, "preprocessed/data_20240514.csv")).set_index("C_cid")

In [ ]:
data

In [ ]:
vocab_C, vocab_C_index = np.unique(data["C_seq_can_smiles"], return_index=True)
vocab_C_cid = data.iloc[vocab_C_index].index
vocab_T, vocab_T_index = np.unique(data["T_seq"], return_index=True)
vocab_T_id = data.iloc[vocab_T_index]["T_id"].values

C_to_index = {vocab_C[i]: i for i in range(len(vocab_C))}
T_to_index = {vocab_T[i]: i for i in range(len(vocab_T))}

# Positive/Negative sampling

In [ ]:
index_train, index_test = train_test_split(pd.Index(np.unique(data.index)), shuffle=True, test_size=0.2)

In [ ]:
data_tr = data.loc[index_train]
data_te = data.loc[index_test]

In [ ]:
data_tr

In [ ]:
data_te

In [ ]:
@nb.njit(['(int8[::1], int8[::1], int8[::1], int8[::1], int8[::1], int8[::1])', '(int8[::1], int8[::1], int8[::1], int8[::1], int8[::1], int8[::1])'], nopython=True, fastmath=True, parallel=True)
def nb_dot(data1, indices1, indptr1, data2, indices2, indptr2):
    nrows1 = indptr1.size-1
    ncols2 = indptr2.size-1
    res = np.zeros((nrows1, ncols2))

    for curr_row1 in nb.prange(nrows1):
        row1_start, row1_end = indptr1[curr_row1], indptr1[curr_row1+1]
        curr_data1 = data1[row1_start:row1_end]
        curr_indices1 = indices1[row1_start:row1_end]

        for curr_row2 in nb.prange(ncols2):
            row2_start, row2_end = indptr2[curr_row2], indptr2[curr_row2+1]
            curr_data2 = data2[row2_start:row2_end]
            curr_indices2 = indices2[row2_start:row2_end]

            for i in range(len(curr_indices1)):
                for j in range(len(curr_indices2)):
                    if curr_indices1[i]==curr_indices2[j]:
                        res[curr_row1,curr_row2] += curr_data1[i] * curr_data2[j]

    return res

def sps_dot_fast(mat1, mat2):
    spsarr1 = sparse.csr_matrix(mat1)
    spsarr2 = sparse.csc_matrix(mat2)
    
    return nb_dot(spsarr1.data.astype(np.int8), spsarr1.indices.astype(np.int8), spsarr1.indptr.astype(np.int8), spsarr2.data.astype(np.int8), spsarr2.indices.astype(np.int8), spsarr2.indptr.astype(np.int8))

In [ ]:
from numba import cuda, float32

In [ ]:
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    
    x, y = cuda.grid(2)
  
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        print(A[x, ty + i * TPB])
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [ ]:
res = np.zeros((CT_adj.shape[0], CT_adj.T.shape[1]))

In [ ]:
cuda_mat1 = cuda.to_device(CT_adj)
cuda_mat2 = cuda.to_device(CT_adj.T)
cuda_res = cuda.to_device(res)

In [ ]:
fast_matmul[8,16](CT_adj, CT_adj.T, res)

In [ ]:
res

In [ ]:
cuda_res.copy_to_host()

In [ ]:

fast_matmul[(32,8),(32,16)](CT_adj, CT_adj.T, res)

In [ ]:
# TEST!
sample_data = data.iloc[:500000]

vocab_C, vocab_C_index = np.unique(sample_data["C_seq_can_smiles"], return_index=True)
vocab_C_cid = sample_data.iloc[vocab_C_index].index
vocab_T, vocab_T_index = np.unique(sample_data["T_seq"], return_index=True)
vocab_T_id = sample_data.iloc[vocab_T_index]["T_id"].values

C_to_index = {vocab_C[i]: i for i in range(len(vocab_C))}
T_to_index = {vocab_T[i]: i for i in range(len(vocab_T))}

In [ ]:
## TEST!
CT_adj = np.zeros((len(vocab_C), len(vocab_T)), dtype=np.float32)
print("Shape of C-T adjacency matrix: ", CT_adj.shape)
index_binding = sample_data.apply(lambda x: (C_to_index[x["C_seq_can_smiles"]], T_to_index[x["T_seq"]]), axis=1)
CT_adj[tuple(np.array(list(zip(*index_binding.values))))] = True

In [ ]:
CT_adj

In [ ]:
def dot_np(A, B):
    C = np.dot(A, B)
    return C

In [ ]:
dot_nb = nb.jit(nb.float32[:,:](nb.float32[:,:], nb.float32[:,:]), nopython=True)(dot_np)

In [ ]:
res = dot_nb(CT_adj, CT_adj.T)

In [ ]:
display(res.sum())

In [ ]:
def chunk_dot(A, B):
    m, n = A.shape
    p, q = B.shape
    assert n == p
    res = np.zeros((m, q), dtype=np.float32)
    for i in range(0,n_chunks):
#         print("For chunk",i)
        start, end = m*i//n_chunks, m*(i+1)//n_chunks
#         print("start:",start,"end:",end)
        res[start:end,:] = A[start:end,:] @ B
        
    return res

In [ ]:
chunk_dot_nb = nb.jit(nb.float32[:,:](nb.float32[:,::1], nb.float32[::1,:]), nopython=True, fastmath=True, parallel=True)(chunk_dot)

In [ ]:
n_chunks=10

In [ ]:
res = chunk_dot_nb(CT_adj, CT_adj.T)

In [ ]:
res.sum()

In [ ]:
def sps_dot(data1, indices1, indptr1, data2, indices2, indptr2):
    nrows1 = indptr1.size-1
    ncols2 = indptr2.size-1
    res = np.zeros((nrows1, ncols2), dtype=np.float32)

    for curr_row1 in nb.prange(nrows1):
        row1_start, row1_end = indptr1[curr_row1], indptr1[curr_row1+1]
        curr_data1 = data1[row1_start:row1_end]
        curr_indices1 = indices1[row1_start:row1_end]

        for curr_row2 in nb.prange(ncols2):
            row2_start, row2_end = indptr2[curr_row2], indptr2[curr_row2+1]
            curr_data2 = data2[row2_start:row2_end]
            curr_indices2 = indices2[row2_start:row2_end]

            for i in range(len(curr_indices1)):
                for j in range(len(curr_indices2)):
                    if curr_indices1[i]==curr_indices2[j]:
                        res[curr_row1,curr_row2] += curr_data1[i] * curr_data2[j]

    return res

def sps_dot_fast(mat1, mat2):
    spsarr1 = sparse.csr_matrix(mat1)
    spsarr2 = sparse.csc_matrix(mat2)
    
    return nb_dot(spsarr1.data.astype(np.float32), spsarr1.indices.astype(np.int8), spsarr1.indptr.astype(np.int8), spsarr2.data.astype(np.float32), spsarr2.indices.astype(np.int8), spsarr2.indptr.astype(np.int8))

In [ ]:
type(spsarr1.data[0] * spsarr2.data[0])

In [ ]:
type(np.zeros((10,12), dtype=np.float32)[0,3])

In [ ]:
spsarr1 = sparse.csr_matrix(CT_adj)
spsarr2 = sparse.csc_matrix(CT_adj.T)

In [ ]:
type(spsarr1.indptr[0])

In [ ]:
nb.int64[:,::1]

In [ ]:
nb_dot = nb.jit(nb.float32[:,:](nb.float32[::1], nb.int64[::1], nb.int64[::1], nb.float32[::1], nb.int64[::1], nb.int64[::1]), nopython=True)(sps_dot)

In [ ]:
res = nb_dot(spsarr1.data.astype(np.float32), spsarr1.indices.astype(np.int64), spsarr1.indptr.astype(np.int64), spsarr2.data.astype(np.float32), spsarr2.indices.astype(np.int64), spsarr2.indptr.astype(np.int64))

In [ ]:
res.sum()

In [ ]:
# TEST!
sample_data = data.iloc[:1000]

vocab_C, vocab_C_index = np.unique(sample_data["C_seq_can_smiles"], return_index=True)
vocab_C_cid = sample_data.iloc[vocab_C_index].index
vocab_T, vocab_T_index = np.unique(sample_data["T_seq"], return_index=True)
vocab_T_id = sample_data.iloc[vocab_T_index]["T_id"].values

C_to_index = {vocab_C[i]: i for i in range(len(vocab_C))}
T_to_index = {vocab_T[i]: i for i in range(len(vocab_T))}

In [ ]:
## TEST!
CT_adj = np.zeros((len(vocab_C), len(vocab_T)), dtype='bool')
print("Shape of C-T adjacency matrix: ", CT_adj.shape)
index_binding = sample_data.apply(lambda x: (C_to_index[x["C_seq_can_smiles"]], T_to_index[x["T_seq"]]), axis=1)
CT_adj[tuple(np.array(list(zip(*index_binding.values))))] = True

In [ ]:
spsarr1 = sparse.csr_matrix(CT_adj)
spsarr2 = sparse.csc_matrix(CT_adj.T)

In [ ]:
res = sps_dot_fast(CT_adj, CT_adj.T)

In [ ]:
spsarr1 = sparse.csr_matrix(CT_adj)
spsarr2 = sparse.csc_matrix(CT_adj.T)

res = nb_dot(spsarr1.data.astype(np.int8), spsarr1.indices.astype(np.int8), spsarr1.indptr.astype(np.int8), spsarr2.data.astype(np.int8), spsarr2.indices.astype(np.int8), spsarr2.indptr.astype(np.int8))

In [ ]:
## Training set에 포함된 pair에 대해서만 인접행렬 업데이트 -> Test set에 포함된 pair 정보는 반영하지 않음
CT_adj = np.zeros((len(vocab_C), len(vocab_T)), dtype='bool')
print("Shape of C-T adjacency matrix: ", CT_adj.shape)
index_binding = data_tr.apply(lambda x: (C_to_index[x["C_seq_can_smiles"]], T_to_index[x["T_seq"]]), axis=1)
CT_adj[tuple(np.array(list(zip(*index_binding.values))))] = True

In [ ]:
CTC_adj = CT_adj.dot(CT_adj.T)
print("Shape of C-T-C adjacency matrix: ", CTC_adj.shape)

In [ ]:
CTCT_adj = np.matmul(CTC_adj, CT_adj)
display(CTCT_adj.shape)

### 경로 테스트

In [ ]:
np.nonzero(CT_adj)

In [ ]:
list(zip(np.nonzero(CT_adj)[0], np.nonzero(CT_adj)[1]))

In [ ]:
np.nonzero(CTCT_adj)

In [ ]:
list(zip(np.nonzero(CTCT_adj)[0], np.nonzero(CTCT_adj)[1]))

In [ ]:
CTCT_adj[42,12]

In [ ]:
vocab_C[42]

In [ ]:
vocab_T[12]

In [ ]:
np.where(vocab_C[42]==sample_set["C_seq_can_smiles"].values)

In [ ]:
sample_set.iloc[66]["C_seq_can_smiles"]

In [ ]:
sample_set.iloc[66]["T_seq"]

In [ ]:
np.where(vocab_T[12]==sample_set["T_seq"].values)

In [ ]:
sample_set.iloc[9]["T_seq"]

In [ ]:
sample_set.loc[sample_set.iloc[9].name].iloc[1]["T_seq"]

## Partition

In [ ]:
def make_pair_index(unzipped_arr):
    return list(zip(unzipped_arr[0], unzipped_arr[1]))

In [ ]:
train_positive = make_pair_index(np.nonzero(CT_adj))

In [ ]:
vocab_C[train_positive[0][0]]

In [ ]:
vocab_T[train_positive[0][1]]

In [ ]:
import os
import sys
from pathlib import Path
import requests
import json
import numpy as np
import pandas as pd
pd.set_option('display.max_rows',200,'display.max_columns',50)
import csv
import time
import pickle
import json

'''
C (ompound) 별 T (arget)의 갯수로 C를 필터링 하는 함수
- 예) 반응하는 단백질 (T)의 갯수가 30개 이상인 화합물 (C)만 필터링
- filter_C_by_num_T(data=reduced_data, N=30)
'''
def insert_CT_code(data):

    pair_code = data.apply(lambda x : str(x['C_cid']) + 'x' + str(x['T_id']), axis=1)
    data.insert(loc=0, column='(C-T)_id', value=list(pair_code))

    return data

def filter_C_by_num_T(data=None, N=30):

    # C (ompound) 별 T (arget)의 갯수로 C를 필터링
    unique_c_ids_n_counts = np.unique(data['C_cid'], return_counts=True)
    unique_c_ids = unique_c_ids_n_counts[0]
    unique_c_counts = unique_c_ids_n_counts[1]

    c_ids_filtering_idx = np.where(unique_c_counts >= N)[0]
    filtered_c_ids = unique_c_ids[c_ids_filtering_idx]
    filtered_c_ids_onehot = data['C_cid'].isin(filtered_c_ids)
    row_index = np.arange(data['C_cid'].shape[0])
    filtered_c_ids_onehot_index = filtered_c_ids_onehot * row_index
    final_filtered_c_ids_index = np.where(filtered_c_ids_onehot_index != 0)[0]

    filtered_data = data.iloc[final_filtered_c_ids_index]

    # CT_code 삽입
    filtered_data = insert_CT_code(filtered_data)

    # 갯수 확인
    num_rows = filtered_data.shape[0]
    num_C = len(np.unique(filtered_data['C_cid']))
    num_T = len(np.unique(filtered_data['T_id']))
    num_CT = len(np.unique(filtered_data['(C-T)_id']))

    # IC50 등에서 값이 달라 (C-T)_id 컬럼에 2개 이상 존재할 경우 가장 첫번째 값만 사용
    filtered_data = filtered_data.groupby('(C-T)_id').first()

    print('Number of T per C >={}\n--> Number of rows : {},\n--> Number of C : {},\n--> Number of T : {}\n--> Number of (C-T) pair : {}'.format(N, num_rows, num_C, num_T, num_CT))
    
    return filtered_data